In [1]:
# Installation & imports
# (À exécuter une seule fois si besoin)
# !pip install langchain-openai langchain gradio python-dotenv

from dotenv import load_dotenv
import os, re, shutil
from pathlib import Path
import xml.etree.ElementTree as ET

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage


In [2]:
# Charge la clé API depuis ENV/env.txt (format: OPENAI_API_KEY=sk-...)
if not load_dotenv(dotenv_path="ENV/env.txt"):
    raise RuntimeError("Impossible de charger ENV/env.txt")

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("Clé API introuvable")

# Modèle (tu peux passer à gpt-4o-mini pour réduire le coût)
llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=api_key)


In [3]:
# Prompt système durci (create_folder + create_file)
system_prompt = """
Tu es un ORCHESTRATEUR IA. Tu reçois une commande en français et tu produis **UNIQUE BLOc** XML strict, parsable par une machine.
N'écris **rien d'autre avant** ni **dans** ce bloc. Après le bloc, tu peux ajouter une courte phrase pour l'utilisateur.

Chemin de base par défaut si relatif : C:\\Users\\jerom\\OneDrive\\Bureau

Actions supportées:
- create_folder
  Champs: path (absolu), make_parents(true/false, def=true), if_exists(skip|error|overwrite, def=skip)
- create_file
  Champs: path (absolu, inclut le nom de fichier), content (texte), if_exists(skip|error|overwrite, def=overwrite), encoding(utf-8 par défaut)

Règles de sécurité:
- Interdire `..`, noms réservés (CON, PRN, AUX, NUL, COM1..9, LPT1..9)
- Remplacer caractères Windows interdits dans segments: <>:"/\\|?*
- Ne jamais ajouter de slash final, conserver l’héritage NTFS implicite
- Si chemin non absolu: préfixer avec le chemin de base

FORMAT OBLIGATOIRE (un seul bloc):
<orchestrator version="1.0">
  <action name="create_folder|create_file">
    <path>CHEMIN_WINDOWS_ABSOLU</path>
    <make_parents>true|false</make_parents>
    <if_exists>skip|error|overwrite</if_exists>
    <content>OPTIONNEL pour create_file</content>
    <encoding>utf-8</encoding>
    <correlation_id>UUID-ou-hash</correlation_id>
    <notes>Brèves notes techniques</notes>
  </action>
</orchestrator>

Exemple:
User: Crée un dossier Projets Lyra 2025\\Rapports sur le Bureau
Output:
<orchestrator version="1.0">
  <action name="create_folder">
    <path>C:\\Users\\jerom\\OneDrive\\Bureau\\Projets Lyra 2025\\Rapports</path>
    <make_parents>true</make_parents>
    <if_exists>skip</if_exists>
    <encoding>utf-8</encoding>
    <correlation_id>...</correlation_id>
    <notes>OK parents</notes>
  </action>
</orchestrator>
"""


In [4]:
# Utilitaires: parse XML & exécution des actions
def parse_orchestrator_block(text: str):
    """
    Extrait le bloc <orchestrator> et retourne un dict paramètres.
    Renvoie None si pas de bloc valide.
    """
    match = re.search(r"(<orchestrator[\s\S]*?</orchestrator>)", text)
    if not match:
        return None
    xml_block = match.group(1)

    root = ET.fromstring(xml_block)
    action = root.find(".//action")
    if action is None:
        return None

    return {
        "action_name": action.attrib.get("name", "").strip(),
        "path": (action.findtext("path") or "").strip(),
        "make_parents": (action.findtext("make_parents", "true").lower() == "true"),
        "if_exists": (action.findtext("if_exists", "skip").lower()),
        "content": action.findtext("content", ""),
        "encoding": action.findtext("encoding", "utf-8"),
        "xml": xml_block,  # utile pour log
    }

def execute_create_folder(params: dict) -> str:
    p = Path(params["path"])
    if p.exists():
        if params["if_exists"] == "skip":
            return f"⏩ Dossier déjà présent : {p}"
        if params["if_exists"] == "error":
            return f"❌ Dossier existe déjà : {p}"
        if params["if_exists"] == "overwrite":
            shutil.rmtree(p)
    p.mkdir(parents=params["make_parents"], exist_ok=True)
    return f"✅ Dossier créé : {p}"

def execute_create_file(params: dict) -> str:
    p = Path(params["path"])
    if p.exists():
        if params["if_exists"] == "skip":
            return f"⏩ Fichier déjà présent : {p}"
        if params["if_exists"] == "error":
            return f"❌ Fichier existe déjà : {p}"
        # overwrite → on continue
    p.parent.mkdir(parents=True, exist_ok=True)
    with open(p, "w", encoding=params.get("encoding", "utf-8")) as f:
        f.write(params.get("content", ""))
    return f"✅ Fichier créé/écrit : {p}"

def execute_from_xml_response(assistant_text: str) -> str:
    """
    Route l'exécution selon l'action détectée dans le XML.
    """
    params = parse_orchestrator_block(assistant_text)
    if not params:
        return "⚠️ Aucune action détectée dans la réponse."
    action = params.get("action_name")
    if action == "create_folder":
        return execute_create_folder(params)
    if action == "create_file":
        return execute_create_file(params)
    return f"⚠️ Action inconnue: {action}"


In [5]:
# ChatPromptTemplate: on injecte système + message utilisateur
# Chaîne LangChain (Système + Humain → LLM)
def run_agent_command(user_command: str):
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_command)
    ]
    resp = llm.invoke(messages)
    return resp.content  # texte complet (XML + éventuel message humain après)


In [6]:
# Cellule 6 — Test rapide (sans UI)
# Exemple 1 : Créer un dossier
out1 = run_agent_command("Crée un dossier 'Demo LangChain\\Essais' sur le Bureau.")
print(out1)
print(execute_from_xml_response(out1))

# Exemple 2 : Créer un fichier
out2 = run_agent_command('Crée un fichier note.txt dans "Demo LangChain" contenant "Bravo ! Ceci est de l\'IA agentique."')
print(out2)
print(execute_from_xml_response(out2))


<orchestrator version="1.0">
  <action name="create_folder">
    <path>C:\Users\jerom\OneDrive\Bureau\Demo LangChain\Essais</path>
    <make_parents>true</make_parents>
    <if_exists>skip</if_exists>
    <encoding>utf-8</encoding>
    <correlation_id>123e4567-e89b-12d3-a456-426614174000</correlation_id>
    <notes>OK parents</notes>
  </action>
</orchestrator>

Le dossier 'Demo LangChain\Essais' sera créé sur le Bureau.
✅ Dossier créé : C:\Users\jerom\OneDrive\Bureau\Demo LangChain\Essais
<orchestrator version="1.0">
  <action name="create_file">
    <path>C:\Users\jerom\OneDrive\Bureau\Demo LangChain\note.txt</path>
    <make_parents>true</make_parents>
    <if_exists>overwrite</if_exists>
    <content>Bravo ! Ceci est de l'IA agentique.</content>
    <encoding>utf-8</encoding>
    <correlation_id>123e4567-e89b-12d3-a456-426614174000</correlation_id>
    <notes>Création du fichier avec contenu spécifié</notes>
  </action>
</orchestrator>

Le fichier note.txt a été configuré pour être

In [7]:
# Interface Gradio minimale (avec LangChain)
import gradio as gr

def gradio_handler(user_input: str):
    # 1) Appel LLM (LangChain)
    assistant_text = run_agent_command(user_input)
    # 2) Exécution locale
    action_result = execute_from_xml_response(assistant_text)
    return assistant_text, action_result

with gr.Blocks() as demo:
    gr.Markdown("## 🖥️ Agent IA Orchestrateur (LangChain) — Dossiers & Fichiers")
    inp = gr.Textbox(label="Commande utilisateur", placeholder='Ex: Crée un fichier Hello.txt contenant "Bonjour !"')
    out_xml = gr.Textbox(label="Sortie Assistant (XML)", lines=10)
    out_exec = gr.Textbox(label="Résultat d’exécution")
    btn = gr.Button("Envoyer")
    btn.click(gradio_handler, inputs=inp, outputs=[out_xml, out_exec])

demo.launch(share=True)  # -> tu peux remettre False si tu veux rester en local


* Running on local URL:  http://127.0.0.1:7875
* Running on public URL: https://5f9a571bf5f5ae8cd4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
